# 📊 Blockchain Transaction Analytics - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on blockchain transaction data stored in BigQuery.

## Objectives
1. **Data Overview**: Understand the structure and quality of raw transaction data
2. **Transaction Analysis**: Distribution of values, gas fees, and timestamps
3. **Wallet Behavior**: Activity patterns, transaction volumes, and counterparties
4. **Anomaly Detection**: Identify potentially suspicious patterns

## Data Sources
- `blockchain_raw.raw_transactions` - Raw Ethereum transactions
- `blockchain_raw.raw_wallets` - Extracted wallet information
- `blockchain_ml.wallet_features` - ML-computed features
- `blockchain_ml.wallet_fraud_scores` - Fraud detection scores


## 1. Setup and Configuration


In [ ]:
# Install required packages (run once if needed)
# !pip install google-cloud-bigquery pandas numpy matplotlib seaborn plotly db-dtypes pyarrow python-dotenv


In [ ]:
# Import libraries
import os
import sys
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from dotenv import load_dotenv

# Configure display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Seaborn palette
sns.set_palette('husl')

print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# Load environment variables
load_dotenv()

# Configuration
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT', 'blockchain-481614')
RAW_DATASET = 'blockchain_raw'
ML_DATASET = 'blockchain_ml'

print(f"Project ID: {PROJECT_ID}")
print(f"Raw Dataset: {RAW_DATASET}")
print(f"ML Dataset: {ML_DATASET}")


In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

def run_query(query: str) -> pd.DataFrame:
    """Execute BigQuery query and return DataFrame."""
    return client.query(query).to_dataframe()

# Test connection
test_query = f"SELECT COUNT(*) as count FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`"
result = run_query(test_query)
print(f"✅ BigQuery connected! Total transactions: {result['count'].iloc[0]:,}")


## 2. Data Overview

Let's start by understanding the structure and quality of our data.


In [ ]:
# Get table information
tables_query = f"""
SELECT 
    table_name,
    ROUND(size_bytes / 1024 / 1024, 2) as size_mb,
    row_count
FROM `{PROJECT_ID}.{RAW_DATASET}.INFORMATION_SCHEMA.TABLE_STORAGE`
ORDER BY table_name
"""

tables_info = run_query(tables_query)
display(tables_info)


In [ ]:
# Sample raw transactions
sample_query = f"""
SELECT *
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
LIMIT 5
"""

sample_df = run_query(sample_query)
print("Raw Transactions Schema:")
print(sample_df.dtypes)
print("\nSample Data:")
display(sample_df)


In [ ]:
# Basic statistics on transactions
stats_query = f"""
SELECT
    COUNT(*) as total_transactions,
    COUNT(DISTINCT from_address) as unique_senders,
    COUNT(DISTINCT to_address) as unique_receivers,
    MIN(transaction_timestamp) as earliest_tx,
    MAX(transaction_timestamp) as latest_tx,
    SUM(CAST(value_eth AS FLOAT64)) as total_value_eth,
    AVG(CAST(value_eth AS FLOAT64)) as avg_value_eth,
    AVG(CAST(gas_used AS FLOAT64)) as avg_gas_used,
    AVG(CAST(gas_price AS FLOAT64) / 1e9) as avg_gas_price_gwei
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
"""

stats_df = run_query(stats_query)
print("📈 Transaction Statistics:")
for col in stats_df.columns:
    val = stats_df[col].iloc[0]
    if isinstance(val, (int, float)) and col not in ['earliest_tx', 'latest_tx']:
        print(f"  {col}: {val:,.2f}")
    else:
        print(f"  {col}: {val}")


## 3. Transaction Value Analysis

Analyze the distribution of transaction values to understand typical behavior and outliers.


In [ ]:
# Transaction value distribution
value_query = f"""
SELECT 
    CAST(value_eth AS FLOAT64) as value_eth,
    CAST(gas_used AS INT64) as gas_used,
    CAST(gas_price AS FLOAT64) / 1e9 as gas_price_gwei,
    transaction_timestamp
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
WHERE value_eth IS NOT NULL
"""

values_df = run_query(value_query)
print(f"Loaded {len(values_df):,} transactions")
values_df.describe()


In [ ]:
# Transaction value distribution visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Value distribution (log scale)
ax1 = axes[0, 0]
non_zero_values = values_df[values_df['value_eth'] > 0]['value_eth']
if len(non_zero_values) > 0:
    ax1.hist(np.log10(non_zero_values + 1e-10), bins=50, edgecolor='black', alpha=0.7, color='#3498db')
ax1.set_xlabel('Log10(Value ETH)')
ax1.set_ylabel('Frequency')
ax1.set_title('Transaction Value Distribution (Log Scale)')
if len(non_zero_values) > 0:
    ax1.axvline(x=np.log10(non_zero_values.median()), color='red', linestyle='--', 
                label=f'Median: {non_zero_values.median():.4f} ETH')
    ax1.legend()

# 2. Zero vs Non-zero transactions
ax2 = axes[0, 1]
zero_count = (values_df['value_eth'] == 0).sum()
non_zero_count = (values_df['value_eth'] > 0).sum()
ax2.pie([zero_count, non_zero_count], labels=['Zero Value', 'Non-Zero Value'], 
        autopct='%1.1f%%', colors=['#e74c3c', '#2ecc71'], explode=(0.05, 0))
ax2.set_title('Zero vs Non-Zero Value Transactions')

# 3. Gas used distribution
ax3 = axes[1, 0]
gas_used = values_df['gas_used'].dropna()
ax3.hist(gas_used, bins=50, edgecolor='black', alpha=0.7, color='#9b59b6')
ax3.set_xlabel('Gas Used')
ax3.set_ylabel('Frequency')
ax3.set_title('Gas Usage Distribution')
ax3.axvline(x=gas_used.median(), color='red', linestyle='--', label=f'Median: {gas_used.median():,.0f}')
ax3.legend()

# 4. Gas price distribution
ax4 = axes[1, 1]
gas_price = values_df['gas_price_gwei'].dropna()
gas_price_filtered = gas_price[gas_price < gas_price.quantile(0.99)]
ax4.hist(gas_price_filtered, bins=50, edgecolor='black', alpha=0.7, color='#f39c12')
ax4.set_xlabel('Gas Price (Gwei)')
ax4.set_ylabel('Frequency')
ax4.set_title('Gas Price Distribution (excluding top 1%)')
ax4.axvline(x=gas_price.median(), color='red', linestyle='--', label=f'Median: {gas_price.median():.2f} Gwei')
ax4.legend()

plt.tight_layout()
plt.savefig('transaction_distributions.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Value percentiles
percentiles = [0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999]
value_percentiles = values_df['value_eth'].quantile(percentiles)

print("📊 Transaction Value Percentiles:")
for p, v in zip(percentiles, value_percentiles):
    print(f"  {p*100:5.1f}%: {v:,.4f} ETH")


## 4. Temporal Analysis

Understand how transaction activity varies over time.


In [ ]:
# Daily transaction volume
daily_query = f"""
SELECT 
    DATE(transaction_timestamp) as tx_date,
    COUNT(*) as tx_count,
    SUM(CAST(value_eth AS FLOAT64)) as total_value,
    AVG(CAST(value_eth AS FLOAT64)) as avg_value,
    COUNT(DISTINCT from_address) as unique_senders,
    AVG(CAST(gas_price AS FLOAT64) / 1e9) as avg_gas_gwei
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
GROUP BY DATE(transaction_timestamp)
ORDER BY tx_date
"""

daily_df = run_query(daily_query)
daily_df['tx_date'] = pd.to_datetime(daily_df['tx_date'])
print(f"Date range: {daily_df['tx_date'].min()} to {daily_df['tx_date'].max()}")
display(daily_df.head(10))


In [ ]:
# Time series visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Transaction count
ax1 = axes[0]
ax1.fill_between(daily_df['tx_date'], daily_df['tx_count'], alpha=0.3, color='#3498db')
ax1.plot(daily_df['tx_date'], daily_df['tx_count'], color='#3498db', linewidth=1)
ax1.set_ylabel('Transaction Count')
ax1.set_title('Daily Transaction Volume Over Time')
ax1.grid(True, alpha=0.3)

# Total value
ax2 = axes[1]
ax2.fill_between(daily_df['tx_date'], daily_df['total_value'], alpha=0.3, color='#2ecc71')
ax2.plot(daily_df['tx_date'], daily_df['total_value'], color='#2ecc71', linewidth=1)
ax2.set_ylabel('Total Value (ETH)')
ax2.set_title('Daily Transaction Value (ETH)')
ax2.grid(True, alpha=0.3)

# Unique senders
ax3 = axes[2]
ax3.fill_between(daily_df['tx_date'], daily_df['unique_senders'], alpha=0.3, color='#9b59b6')
ax3.plot(daily_df['tx_date'], daily_df['unique_senders'], color='#9b59b6', linewidth=1)
ax3.set_ylabel('Unique Senders')
ax3.set_xlabel('Date')
ax3.set_title('Daily Unique Active Senders')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('temporal_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Hour of day and day of week analysis
hourly_query = f"""
SELECT 
    EXTRACT(HOUR FROM transaction_timestamp) as hour_of_day,
    COUNT(*) as tx_count,
    SUM(CAST(value_eth AS FLOAT64)) as total_value
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
GROUP BY hour_of_day
ORDER BY hour_of_day
"""

hourly_df = run_query(hourly_query)

dow_query = f"""
SELECT 
    EXTRACT(DAYOFWEEK FROM transaction_timestamp) as day_of_week,
    COUNT(*) as tx_count,
    SUM(CAST(value_eth AS FLOAT64)) as total_value
FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
GROUP BY day_of_week
ORDER BY day_of_week
"""

dow_df = run_query(dow_query)
dow_df['day_name'] = dow_df['day_of_week'].map({1: 'Sun', 2: 'Mon', 3: 'Tue', 4: 'Wed', 5: 'Thu', 6: 'Fri', 7: 'Sat'})


In [ ]:
# Hour and day of week visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hour of day
ax1 = axes[0]
ax1.bar(hourly_df['hour_of_day'], hourly_df['tx_count'], color='#3498db', alpha=0.8)
ax1.set_xlabel('Hour of Day (UTC)')
ax1.set_ylabel('Transaction Count')
ax1.set_title('Transaction Activity by Hour of Day')
ax1.set_xticks(range(0, 24, 2))

# Day of week
ax2 = axes[1]
colors = ['#e74c3c' if d in ['Sat', 'Sun'] else '#3498db' for d in dow_df['day_name']]
ax2.bar(dow_df['day_name'], dow_df['tx_count'], color=colors, alpha=0.8)
ax2.set_xlabel('Day of Week')
ax2.set_ylabel('Transaction Count')
ax2.set_title('Transaction Activity by Day of Week (Weekend in Red)')

plt.tight_layout()
plt.savefig('time_patterns.png', dpi=150, bbox_inches='tight')
plt.show()


## 5. Wallet Activity Analysis

Examine wallet behavior patterns to identify high-activity and potentially suspicious wallets.


In [ ]:
# Top wallets by transaction count
top_wallets_query = f"""
WITH wallet_activity AS (
    SELECT 
        from_address as wallet,
        COUNT(*) as tx_count,
        SUM(CAST(value_eth AS FLOAT64)) as total_value,
        'sender' as role
    FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
    GROUP BY from_address
    
    UNION ALL
    
    SELECT 
        to_address as wallet,
        COUNT(*) as tx_count,
        SUM(CAST(value_eth AS FLOAT64)) as total_value,
        'receiver' as role
    FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
    GROUP BY to_address
)
SELECT 
    wallet,
    SUM(tx_count) as total_tx,
    SUM(total_value) as total_value,
    SUM(CASE WHEN role = 'sender' THEN tx_count ELSE 0 END) as sent_tx,
    SUM(CASE WHEN role = 'receiver' THEN tx_count ELSE 0 END) as received_tx
FROM wallet_activity
WHERE wallet IS NOT NULL
GROUP BY wallet
ORDER BY total_tx DESC
LIMIT 20
"""

top_wallets_df = run_query(top_wallets_query)
print("🔝 Top 20 Wallets by Transaction Count:")
display(top_wallets_df)


In [ ]:
# Visualize top wallets
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Top wallets by transaction count
ax1 = axes[0]
top_10 = top_wallets_df.head(10)
y_pos = range(len(top_10))
ax1.barh(y_pos, top_10['total_tx'], color='#2ecc71', alpha=0.8)
ax1.set_yticks(y_pos)
ax1.set_yticklabels([f"{w[:8]}...{w[-4:]}" for w in top_10['wallet']])
ax1.set_xlabel('Transaction Count')
ax1.set_title('Top 10 Wallets by Transaction Count')
ax1.invert_yaxis()

# Top wallets by value
ax2 = axes[1]
top_by_value = top_wallets_df.sort_values('total_value', ascending=False).head(10)
y_pos = range(len(top_by_value))
ax2.barh(y_pos, top_by_value['total_value'], color='#3498db', alpha=0.8)
ax2.set_yticks(y_pos)
ax2.set_yticklabels([f"{w[:8]}...{w[-4:]}" for w in top_by_value['wallet']])
ax2.set_xlabel('Total Value (ETH)')
ax2.set_title('Top 10 Wallets by Total Value')
ax2.invert_yaxis()

plt.tight_layout()
plt.savefig('wallet_activity.png', dpi=150, bbox_inches='tight')
plt.show()


## 6. Fraud Score Analysis

If the ML pipeline has been run, analyze the computed fraud scores.


In [ ]:
# Load fraud scores (from ML pipeline)
try:
    scores_query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{ML_DATASET}.wallet_fraud_scores`
    """
    scores_df = run_query(scores_query)
    print(f"✅ Loaded fraud scores for {len(scores_df):,} wallets")
    scores_available = True
    display(scores_df.head())
except Exception as e:
    print(f"⚠️ Fraud scores table not found. Run the ML pipeline first.")
    print(f"   Error: {e}")
    scores_available = False


In [ ]:
# Fraud score distribution visualization
if scores_available:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1 = axes[0]
    ax1.hist(scores_df['fraud_score'], bins=50, edgecolor='black', alpha=0.7, color='#e74c3c')
    ax1.axvline(x=0.4, color='orange', linestyle='--', linewidth=2, label='Medium Risk (0.4)')
    ax1.axvline(x=0.7, color='red', linestyle='--', linewidth=2, label='High Risk (0.7)')
    ax1.set_xlabel('Fraud Score')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Fraud Score Distribution')
    ax1.legend()
    
    # Risk category pie chart
    ax2 = axes[1]
    risk_counts = scores_df['risk_category'].value_counts()
    colors_map = {'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c', 'critical': '#8e44ad'}
    colors = [colors_map.get(c, '#95a5a6') for c in risk_counts.index]
    ax2.pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%', colors=colors)
    ax2.set_title('Risk Category Distribution')
    
    plt.tight_layout()
    plt.savefig('fraud_scores.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Risk Category Counts:")
    print(risk_counts)
else:
    print("Skipping fraud score visualization - ML pipeline not run yet")


In [ ]:
# High risk wallets analysis
if scores_available:
    high_risk = scores_df[scores_df['risk_category'].isin(['high', 'critical'])].sort_values('fraud_score', ascending=False)
    print(f"🚨 High/Critical Risk Wallets: {len(high_risk)}")
    display(high_risk.head(10))
else:
    print("Skipping high risk analysis - ML pipeline not run yet")


## 7. Key Insights Summary

Summarize the key findings from this exploratory analysis.


In [ ]:
# Generate summary insights
print("="*60)
print("📋 EDA SUMMARY")
print("="*60)

# Data overview
print("\n📊 DATA OVERVIEW:")
print(f"  - Total transactions analyzed: {len(values_df):,}")
print(f"  - Date range: {daily_df['tx_date'].min().date()} to {daily_df['tx_date'].max().date()}")
print(f"  - Total value transferred: {values_df['value_eth'].sum():,.2f} ETH")

# Transaction patterns
print("\n💹 TRANSACTION PATTERNS:")
print(f"  - Zero-value transactions: {(values_df['value_eth'] == 0).sum():,} ({(values_df['value_eth'] == 0).mean()*100:.1f}%)")
print(f"  - Median transaction value: {values_df['value_eth'].median():.4f} ETH")
print(f"  - Average gas used: {values_df['gas_used'].mean():,.0f}")

# Wallet activity
print("\n👛 WALLET ACTIVITY:")
print(f"  - Most active wallet: {top_wallets_df.iloc[0]['wallet'][:16]}...")
print(f"  - Most active wallet txs: {top_wallets_df.iloc[0]['total_tx']:,}")

# ML insights (if available)
if scores_available:
    print("\n🤖 ML FRAUD DETECTION:")
    print(f"  - Wallets scored: {len(scores_df):,}")
    print(f"  - High/Critical risk: {len(high_risk):,} ({len(high_risk)/len(scores_df)*100:.1f}%)")
    print(f"  - Average fraud score: {scores_df['fraud_score'].mean():.3f}")

print("\n" + "="*60)


## 📁 Saved Outputs

The following visualization files were generated:
- `transaction_distributions.png` - Value and gas distributions
- `temporal_analysis.png` - Time series of transaction activity
- `time_patterns.png` - Hour of day and day of week patterns
- `wallet_activity.png` - Top wallet activity
- `fraud_scores.png` - Fraud score distributions (if ML pipeline was run)

---

**Next Steps:**
1. Run the ML notebook (`02_fraud_detection_ml.ipynb`) to train fraud detection models
2. Review high-risk wallets identified by the analysis
3. Investigate temporal anomalies in transaction patterns
